<a href="https://colab.research.google.com/github/RasonYudha4/Medicines-Recommender/blob/main/Medicine_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing every modules and data that are needed**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re

from gensim.models import Word2Vec
from tqdm import tqdm

# **Data Preprocessing**

In [ ]:
data1.head()

,id,name,price(₹),Is_discontinued,manufacturer_name,type,pack_size_label,short_composition1,short_composition2
0,1,Augmentin 625 Duo Tablet,223.42,False,Glaxo SmithKline Pharmaceuticals Ltd,allopathy,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg)
1,2,Azithral 500 Tablet,132.36,False,Alembic Pharmaceuticals Ltd,allopathy,strip of 5 tablets,Azithromycin (500mg),NaN
2,3,Ascoril LS Syrup,118.00,False,Glenmark Pharmaceuticals Ltd,allopathy,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml)
3,4,Allegra 120mg Tablet,218.81,False,Sanofi India Ltd,allopathy,strip of 10 tablets,Fexofenadine (120mg),NaN
4,5,Avil 25 Tablet,10.96,False,Sanofi India Ltd,allopathy,strip of 15 tablets,Pheniramine (25mg),NaN


In [ ]:
data2.head()

,id,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,sideEffect2,...,sideEffect41,use0,use1,use2,use3,use4,Chemical Class,Habit Forming,Therapeutic Class,Action Class
0,1,augmentin 625 duo tablet,Penciclav 500 mg/125 mg Tablet,Moxikind-CV 625 Tablet,Moxiforce-CV 625 Tablet,Fightox 625 Tablet,Novamox CV 625mg Tablet,Vomiting,Nausea,Diarrhea,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,NaN,No,ANTI INFECTIVES,NaN
1,2,azithral 500 tablet,Zithrocare 500mg Tablet,Azax 500 Tablet,Zady 500 Tablet,Cazithro 500mg Tablet,Trulimax 500mg Tablet,Vomiting,Nausea,Abdominal pain,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Macrolides,No,ANTI INFECTIVES,Macrolides
2,3,ascoril ls syrup,Solvin LS Syrup,Ambrodil-LX Syrup,Zerotuss XP Syrup,Capex LS Syrup,Broxum LS Syrup,Nausea,Vomiting,Diarrhea,...,NaN,Treatment of Cough with mucus,NaN,NaN,NaN,NaN,NaN,No,RESPIRATORY,NaN
3,4,allegra 120mg tablet,Lcfex Tablet,Etofex 120mg Tablet,Nexofex 120mg Tablet,Fexise 120mg Tablet,Histafree 120 Tablet,Headache,Drowsiness,Dizziness,...,NaN,Treatment of Sneezing and runny nose due to al...,Treatment of Allergic conditions,NaN,NaN,NaN,Diphenylmethane Derivative,No,RESPIRATORY,H1 Antihistaminics (second Generation)
4,5,avil 25 tablet,Eralet 25mg Tablet,NaN,NaN,NaN,NaN,Sleepiness,Dryness in mouth,NaN,...,NaN,Treatment of Allergic conditions,NaN,NaN,NaN,NaN,Pyridines Derivatives,No,RESPIRATORY,H1 Antihistaminics (First Generation)


## Data Cleansing


**Check and delete any duplication of data1 and data2 (general)**

In [ ]:
data1 = data1.drop(columns=['id'])
data2 = data2.drop(columns=['id'])

In [ ]:
duplicates1 = data1.duplicated()
num_duplicates1 = duplicates1.sum()
print(f'Number of duplication in data 1: {num_duplicates1}')

duplicates2 = data2.duplicated()
num_duplicates2 = duplicates2.sum()
print(f'Number of duplication in data 2: {num_duplicates2}')

Number of duplication in data 1: 5
Number of duplication in data 2: 24204


In [ ]:
data1 = data1.drop_duplicates()
data2 = data2.drop_duplicates()

**Check and delete any duplication of data1 and data2 (based on the name column)**

### Data 1 Cleansing

#### 'Name' Column

In [ ]:
# Check the number of unique value in data 1 based on 'name' column
name_counts = data1['name'].value_counts()

# Only show when the unique value is greater than 1
name_counts_filtered = name_counts[name_counts > 1]
print(name_counts_filtered)

name
NS 0.9% Infusion                  12
Ringer Lactate Infusion            7
Luligee Cream                      5
Azilife 500mg Tablet               5
Leflox 500mg Tablet                5
                                  ..
Ultravist 300mg I/ml Injection     2
Intor 80mg Tablet                  2
Orange CS Syrup                    2
Lodep 25mg Tablet                  2
Ceriz 5mg Syrup                    2
Name: count, Length: 4283, dtype: int64


In [ ]:
data1[data1['name'] == "Ringer Lactate Infusion"]

,name,price(₹),Is_discontinued,manufacturer_name,type,pack_size_label,short_composition1,short_composition2
186117,Ringer Lactate Infusion,22.50,False,Baxter India Pvt Ltd,allopathy,bottle of 500 ml Infusion,Sodium Chloride (0.600gm),Sodium Lactate (0.320gm)
188582,Ringer Lactate Infusion,46.11,False,Albert David Ltd,allopathy,bottle of 500 ml Infusion,Sodium Chloride (0.600gm),Sodium Lactate (0.320gm)
189321,Ringer Lactate Infusion,49.72,False,Otsuka Pharmaceutical India Pvt Ltd,allopathy,bottle of 500 ml Infusion,Sodium Chloride (0.6gm),Sodium Lactate (0.32gm)
189803,Ringer Lactate Infusion,49.73,False,Parenteral Drugs India Ltd,allopathy,bottle of 500 ml Infusion,Sodium Chloride (600mg),Sodium Lactate (320mg)
192464,Ringer Lactate Infusion,37.50,False,Otsuka Pharmaceutical India Pvt Ltd,allopathy,bottle of 1000 ml Infusion,Sodium Chloride (0.6gm),Sodium Lactate (0.32gm)
194320,Ringer Lactate Infusion,23.33,False,Parenteral Drugs India Ltd,allopathy,bottle of 1000 ml Infusion,Sodium Chloride (0.6gm),Sodium Lactate (0.32gm)
195433,Ringer Lactate Infusion,29.81,False,Nirlife Healthcare,allopathy,bottle of 500 ml Infusion,Sodium Chloride (0.600gm),Sodium Lactate (0.320gm)


Since the duplication happens because of the different manufacturer name, therefore the data is still valid to use

#### 'Pack_size_label'

In [ ]:
form=[]
count={}
for i in data1.pack_size_label:
    words=i.split()
    x=words[-1].lower()
    if len(x)<=2:
        x=words[-2].lower()
    if x in form:
        count[x]+=1
    else:
        count[x]=1
        form.append(x)

sorted_count = dict(sorted(count.items(), key=lambda x:x[0]))
sorted_count

{'aerosol': 1,
 'alicaps': 1,
 'ampules': 1,
 'aquanase': 2,
 'autohaler': 3,
 'autopen': 1,
 'bandage': 2,
 'bar': 3,
 'bio-adhesive': 2,
 'bottle': 1,
 'bottles': 16,
 'caplets': 5,
 'capsule': 6275,
 'capsules': 15824,
 'captabs': 2,
 'cartridge': 1,
 'cartriges': 6,
 'cleanser': 1,
 'combikit': 17,
 'combipack': 12,
 'conditioner': 2,
 'cream': 5067,
 'creams': 4,
 'diskettes': 3,
 'drop': 3859,
 'drops': 1423,
 'elixir': 3,
 'elixirs': 1,
 'emulsion': 22,
 'emulsions': 1,
 'enema': 7,
 'expectorant': 338,
 'film': 1,
 'foam': 9,
 'gargle': 53,
 'gel': 1686,
 'gels': 1,
 'grains': 2,
 'granules': 127,
 'gum': 2,
 'gummies': 2,
 'gums': 15,
 'husk': 6,
 'infusion': 902,
 'infusions': 1,
 'inhalation': 29,
 'inhaler': 267,
 'injection': 31907,
 'injections': 137,
 'instacap': 1,
 'instacaps': 4,
 'jelly': 11,
 'kit': 248,
 'lacquer': 9,
 'linctus': 19,
 'liniment': 7,
 'liquid': 163,
 'lotion': 1021,
 'lotions': 1,
 'lozenges': 25,
 'multihaler': 7,
 'nebulisers': 1,
 'nexcaps': 4,
 

In the 'pack_size_label' we can see that there are some plural issue on the data such as bottle and bottles which will serve the same meaning.

Therefore, we need to manipulate the data by combining both plural and singular data and label it as singular. This will help us later on during the feature embedding

In [ ]:
# Eliminating the plural issue then combine it with singular
final_count=sorted_count.copy()
for key,value in sorted_count.items():
    test=(key+'s') in final_count
    if test:
        final_count[key]+=final_count[key+'s']
        del final_count[key+'s']
        form.remove(key+'s')

final_count = dict(sorted(final_count.items(), key=lambda x:x[1], reverse=True))
final_count

{'tablet': 152552,
 'injection': 32044,
 'capsule': 22099,
 'syrup': 16747,
 'suspension': 9178,
 'drop': 5282,
 'cream': 5071,
 'solution': 1716,
 'gel': 1687,
 'ointment': 1669,
 'lotion': 1022,
 'infusion': 903,
 'soap': 526,
 'powder': 373,
 'spray': 366,
 'expectorant': 338,
 'shampoo': 271,
 'inhaler': 267,
 'kit': 248,
 'wash': 195,
 'liquid': 163,
 'granules': 127,
 'respules': 122,
 'suppositories': 76,
 'sachet': 70,
 'rotacaps': 66,
 'strip': 64,
 'gargle': 53,
 'patches': 47,
 'syringe': 35,
 'paint': 34,
 'respicaps': 33,
 'suppository': 32,
 'inhalation': 29,
 'lozenges': 25,
 'emulsion': 23,
 'transcaps': 23,
 'pessaries': 20,
 'transhaler': 20,
 'linctus': 19,
 'paste': 18,
 'patch': 18,
 'bottle': 17,
 'combikit': 17,
 'gum': 17,
 'toothpaste': 16,
 'combipack': 12,
 'transpules': 12,
 'jelly': 11,
 'serum': 10,
 'foam': 9,
 'lacquer': 9,
 'scrub': 9,
 'tabcaps': 9,
 'novocart': 8,
 'enema': 7,
 'liniment': 7,
 'multihaler': 7,
 'rheocap': 7,
 'cartriges': 6,
 'husk': 

### Data 2 Cleansing


#### 'Name' Column

In [ ]:
# Check the number of unique value in data 1 based on 'name' column
name_counts2 = data2['name'].value_counts()

# Only show when the unique value is greater than 1
name_counts_filtered2 = name_counts2[name_counts2 > 1]
print(name_counts_filtered2)

name
ringer lactate infusion       4
cefpoxim dry syrup            4
azicon 500mg tablet           3
loxin 500mg tablet            3
kidivent-ls syrup             3
                             ..
fycare-d 30mg/20mg capsule    2
mytrip 25mg tablet            2
amcard 5 tablet               2
weldec 50mg injection         2
cypotin syrup                 2
Name: count, Length: 1165, dtype: int64


In [ ]:
data2[data2['name'] == "ringer lactate infusion"]

,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,sideEffect2,sideEffect3,...,sideEffect41,use0,use1,use2,use3,use4,Chemical Class,Habit Forming,Therapeutic Class,Action Class
199320,ringer lactate infusion,Ringer Lactate Infusion,Ringer Lactate IP Poly Infusion,RL Infusion,Ringer Lactate Infusion,NaN,No common side effects seen,NaN,NaN,NaN,...,NaN,Short term fluid replacement after trauma,NaN,NaN,NaN,NaN,NaN,No,BLOOD RELATED,NaN
201811,ringer lactate infusion,Ringer Lactate Infusion,Ringer Lactate Infusion,Ringer Lactate IP Poly Infusion,RL Infusion,NaN,No common side effects seen,NaN,NaN,NaN,...,NaN,Short term fluid replacement after trauma,NaN,NaN,NaN,NaN,NaN,No,BLOOD RELATED,NaN
202568,ringer lactate infusion,Ringer Lactate Infusion,RINGER LACTATE INFUSION,RINGER LACTATE INFUSION,NaN,NaN,No common side effects seen,NaN,NaN,NaN,...,NaN,Short term fluid replacement after trauma,NaN,NaN,NaN,NaN,NaN,No,BLOOD RELATED,NaN
203065,ringer lactate infusion,NaN,NaN,NaN,NaN,NaN,No common side effects seen,NaN,NaN,NaN,...,NaN,Short term fluid replacement after trauma,NaN,NaN,NaN,NaN,NaN,No,BLOOD RELATED,NaN


In [ ]:
data2['null_count'] = data2.isnull().sum(axis=1)
data2

,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,sideEffect2,sideEffect3,...,use0,use1,use2,use3,use4,Chemical Class,Habit Forming,Therapeutic Class,Action Class,null_count
0,augmentin 625 duo tablet,Penciclav 500 mg/125 mg Tablet,Moxikind-CV 625 Tablet,Moxiforce-CV 625 Tablet,Fightox 625 Tablet,Novamox CV 625mg Tablet,Vomiting,Nausea,Diarrhea,NaN,...,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,NaN,No,ANTI INFECTIVES,NaN,45
1,azithral 500 tablet,Zithrocare 500mg Tablet,Azax 500 Tablet,Zady 500 Tablet,Cazithro 500mg Tablet,Trulimax 500mg Tablet,Vomiting,Nausea,Abdominal pain,Diarrhea,...,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Macrolides,No,ANTI INFECTIVES,Macrolides,42
2,ascoril ls syrup,Solvin LS Syrup,Ambrodil-LX Syrup,Zerotuss XP Syrup,Capex LS Syrup,Broxum LS Syrup,Nausea,Vomiting,Diarrhea,Upset stomach,...,Treatment of Cough with mucus,NaN,NaN,NaN,NaN,NaN,No,RESPIRATORY,NaN,34
3,allegra 120mg tablet,Lcfex Tablet,Etofex 120mg Tablet,Nexofex 120mg Tablet,Fexise 120mg Tablet,Histafree 120 Tablet,Headache,Drowsiness,Dizziness,Nausea,...,Treatment of Sneezing and runny nose due to al...,Treatment of Allergic conditions,NaN,NaN,NaN,Diphenylmethane Derivative,No,RESPIRATORY,H1 Antihistaminics (second Generation),41
4,avil 25 tablet,Eralet 25mg Tablet,NaN,NaN,NaN,NaN,Sleepiness,Dryness in mouth,NaN,NaN,...,Treatment of Allergic conditions,NaN,NaN,NaN,NaN,Pyridines Derivatives,No,RESPIRATORY,H1 Antihistaminics (First Generation),48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248210,ziyapod 100mg oral suspension,Qc Pod Dry Syrup,Actipod 50 Dry Syrup Orange-Lemon,Tilpod 100 Dry Syrup,Jadupox-DS Dry Syrup,Podwel 100 Dry Syrup Orange,Rash,Nausea,Diarrhea,NaN,...,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Broad Spectrum (Third & fourth generation ceph...,No,ANTI INFECTIVES,Cephalosporins: 3 generation,43
248212,zemhart 30mg tablet,Dilzex 30mg Tablet,Diltiaz 30mg Tablet,Triazem 30mg Tablet,Dilti 30mg Tablet,DZ 30mg Tablet,Headache,Constipation,Dizziness,Fatigue,...,Treatment of Hypertension (high blood pressure),Treatment of Angina (heart-related chest pain),Treatment of Arrhythmia,NaN,NaN,Benzothiazepine derivative,No,CARDIAC,Calcium channel blockers- Nondihydropyridines,36
248215,zivex 25mg tablet,HD Zine 25mg Tablet,Hydrocas 25mg Tablet,Hyzox 25 Tablet,Hydil 25mg Tablet,Zyzine 25mg Tablet,Sedation,Nausea,Vomiting,Upset stomach,...,Treatment of Anxiety,Treatment of Skin conditions with inflammation...,NaN,NaN,NaN,Piperazine Derivative,No,RESPIRATORY,H1 Antihistaminics (First Generation),40
248216,zi fast 500mg injection,Zycin 500mg Injection,Aziwok 500mg Injection,Azirow 500mg Injection,Toracin 500mg Injection,Azymed 500mg Injection,"Injection site reactions (pain, swelling, redn...",Vomiting,Nausea,Abdominal pain,...,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Macrolides,No,ANTI INFECTIVES,Macrolides,41


In [ ]:
data2_sorted = data2.sort_values(by=['name', 'null_count'])
data2_sorted

,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,sideEffect2,sideEffect3,...,use0,use1,use2,use3,use4,Chemical Class,Habit Forming,Therapeutic Class,Action Class,null_count
996,Alto's citrasol liquid,Cadicitron 1.4gm/5ml Syrup,Acrocitron 1.4gm/5ml Syrup,Alkacit Syrup,Alkawin 1.4gm/5ml Syrup,Citodac Syrup,Stomach pain,Tiredness,Diarrhea,Nausea,...,Treatment of Gout,Treatment of Kidney stone,NaN,NaN,NaN,Carboxylic acid derivative,No,UROLOGY,Uricosuric agent-gout,39
6434,a 1 5mg tablet,StayHappi Amlodipine 5mg Tablet,Amlip 5 Tablet,Avacard 5mg Tablet,Amcard 5 Tablet,Amlong Tablet,Headache,Fatigue,Nausea,Abdominal pain,...,Treatment of Hypertension (high blood pressure),Prevention of Angina (heart-related chest pain),NaN,NaN,NaN,Dihydropyridinecarboxylic acids derivatives,No,CARDIAC,Calcium channel blockers- Dihydropyridines (DHP),40
6139,a 250 suspension,Rakbi 250mg Suspension,Alcocin Suspension,Tempros Suspension,Mormol 250mg Suspension,Jetmol Suspension,Indigestion,Stomach pain,Nausea,Vomiting,...,Pain relief,Treatment of Fever,NaN,NaN,NaN,P-Aminophenol Derivative,No,PAIN ANALGESICS,Analgesic & Antipyretic-PCM,41
13016,a 3 100 mg/500 mg tablet,Nimsa Plus 100mg/500mg Tablet,Nicet Tablet,Nesu P 100mg/500mg Tablet,Nimeson P 100 mg/500 mg Tablet,Tromanil Plus 100 mg/500 mg Tablet,Nausea,Vomiting,Diarrhea,Increased liver enzymes,...,Pain relief,NaN,NaN,NaN,NaN,NaN,No,PAIN ANALGESICS,NaN,44
14692,a arti 60mg injection,Arteross 60mg Injection,Falciart 60mg Injection,Falsitis AR Injection,Leonate 60mg Injection,Neosunate 60mg Injection,Headache,Dizziness,Weakness,Loss of appetite,...,Malaria,NaN,NaN,NaN,NaN,Sesquiterpene lactones,No,ANTI MALARIALS,Antimalarial- Artemisinin and derivatives,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248112,zyxtil 500mg tablet,Cefoxim 500 Tablet,Zefu 500 Tablet,Bigcef 500 Tablet,Cefakind 500 Tablet,Pulmocef 500 Tablet,Rash,Vomiting,Allergic reaction,Increased liver enzymes,...,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Intermediate spectrum {Second generation cepha...,No,ANTI INFECTIVES,Cephalosporins: 2nd generation,40
241389,zyzer syrup,Trisotis Plus Syrup,Anzicyp Syrup,Mecyp Syrup,Cytadine-TS Syrup,Pneucyprodin T Syrup,Constipation,Dryness in mouth,Drowsiness,Sleepiness,...,Appetite stimulant,NaN,NaN,NaN,NaN,NaN,No,VITAMINS MINERALS NUTRIENTS,NaN,43
243012,zyzine 25mg tablet,HD Zine 25mg Tablet,Hyzox 25 Tablet,Hydil 25mg Tablet,Hizone 25mg Tablet,Vistarax 25mg Tablet,Sedation,Nausea,Vomiting,Upset stomach,...,Treatment of Anxiety,Treatment of Skin conditions with inflammation...,NaN,NaN,NaN,Piperazine Derivative,No,RESPIRATORY,H1 Antihistaminics (First Generation),40
241756,zyzolide 600mg tablet,Warline 600mg Tablet,Linokem 600 Tablet,Lizoforce 600 Tablet,Linid Tablet,Megazolid 600 Tablet,Vomiting,Headache,Nausea,"Decreased blood cells (red cells, white cells,...",...,Treatment of Severe bacterial infections,NaN,NaN,NaN,NaN,Oxazolidinone derivative,No,ANTI INFECTIVES,Oxazolidinone,41


In [ ]:
data2_cleaned = data2_sorted.drop_duplicates(subset=['name'], keep='first')
data2_cleaned

,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,sideEffect2,sideEffect3,...,use0,use1,use2,use3,use4,Chemical Class,Habit Forming,Therapeutic Class,Action Class,null_count
996,Alto's citrasol liquid,Cadicitron 1.4gm/5ml Syrup,Acrocitron 1.4gm/5ml Syrup,Alkacit Syrup,Alkawin 1.4gm/5ml Syrup,Citodac Syrup,Stomach pain,Tiredness,Diarrhea,Nausea,...,Treatment of Gout,Treatment of Kidney stone,NaN,NaN,NaN,Carboxylic acid derivative,No,UROLOGY,Uricosuric agent-gout,39
6434,a 1 5mg tablet,StayHappi Amlodipine 5mg Tablet,Amlip 5 Tablet,Avacard 5mg Tablet,Amcard 5 Tablet,Amlong Tablet,Headache,Fatigue,Nausea,Abdominal pain,...,Treatment of Hypertension (high blood pressure),Prevention of Angina (heart-related chest pain),NaN,NaN,NaN,Dihydropyridinecarboxylic acids derivatives,No,CARDIAC,Calcium channel blockers- Dihydropyridines (DHP),40
6139,a 250 suspension,Rakbi 250mg Suspension,Alcocin Suspension,Tempros Suspension,Mormol 250mg Suspension,Jetmol Suspension,Indigestion,Stomach pain,Nausea,Vomiting,...,Pain relief,Treatment of Fever,NaN,NaN,NaN,P-Aminophenol Derivative,No,PAIN ANALGESICS,Analgesic & Antipyretic-PCM,41
13016,a 3 100 mg/500 mg tablet,Nimsa Plus 100mg/500mg Tablet,Nicet Tablet,Nesu P 100mg/500mg Tablet,Nimeson P 100 mg/500 mg Tablet,Tromanil Plus 100 mg/500 mg Tablet,Nausea,Vomiting,Diarrhea,Increased liver enzymes,...,Pain relief,NaN,NaN,NaN,NaN,NaN,No,PAIN ANALGESICS,NaN,44
14692,a arti 60mg injection,Arteross 60mg Injection,Falciart 60mg Injection,Falsitis AR Injection,Leonate 60mg Injection,Neosunate 60mg Injection,Headache,Dizziness,Weakness,Loss of appetite,...,Malaria,NaN,NaN,NaN,NaN,Sesquiterpene lactones,No,ANTI MALARIALS,Antimalarial- Artemisinin and derivatives,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248112,zyxtil 500mg tablet,Cefoxim 500 Tablet,Zefu 500 Tablet,Bigcef 500 Tablet,Cefakind 500 Tablet,Pulmocef 500 Tablet,Rash,Vomiting,Allergic reaction,Increased liver enzymes,...,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Intermediate spectrum {Second generation cepha...,No,ANTI INFECTIVES,Cephalosporins: 2nd generation,40
241389,zyzer syrup,Trisotis Plus Syrup,Anzicyp Syrup,Mecyp Syrup,Cytadine-TS Syrup,Pneucyprodin T Syrup,Constipation,Dryness in mouth,Drowsiness,Sleepiness,...,Appetite stimulant,NaN,NaN,NaN,NaN,NaN,No,VITAMINS MINERALS NUTRIENTS,NaN,43
243012,zyzine 25mg tablet,HD Zine 25mg Tablet,Hyzox 25 Tablet,Hydil 25mg Tablet,Hizone 25mg Tablet,Vistarax 25mg Tablet,Sedation,Nausea,Vomiting,Upset stomach,...,Treatment of Anxiety,Treatment of Skin conditions with inflammation...,NaN,NaN,NaN,Piperazine Derivative,No,RESPIRATORY,H1 Antihistaminics (First Generation),40
241756,zyzolide 600mg tablet,Warline 600mg Tablet,Linokem 600 Tablet,Lizoforce 600 Tablet,Linid Tablet,Megazolid 600 Tablet,Vomiting,Headache,Nausea,"Decreased blood cells (red cells, white cells,...",...,Treatment of Severe bacterial infections,NaN,NaN,NaN,NaN,Oxazolidinone derivative,No,ANTI INFECTIVES,Oxazolidinone,41


In [ ]:
data2 = data2_cleaned.drop(columns=['null_count'])

#### 'Use' Column

In [ ]:
data2['use0'].value_counts()

,count
use0,
Treatment of Bacterial infections,34386
Pain relief,20570
Treatment of Type 2 diabetes mellitus,10093
Bacterial infections,9667
Treatment of Gastroesophageal reflux disease (Acid reflux),8942
...,...
Prevention of HPV-associated cancers,1
Benzodiazepine overdose,1
Diabetic macular edema (accumulation of fluid in eye of diabetic patients),1


Looking at it carefully, there are some sentences that serves the same meaning only with different word structure.

(ex. 'Treatment of Bacterial infections' with 'Bacterial infections')

Therefore, we can do data cleansing by eliminating the verbs that serve the same meaning

In [ ]:
def formatted_sentences(text):
    text = text.lower()

    # Eliminating the verbs that has the simillar meaning
    text = re.sub(r'\b(treatment|prevention)\s*(of|and prevention of)?\b', '', text)

    return text.strip()

data2['use0'] = [formatted_sentences(text) if isinstance(text, str) else text for text in data2['use0']]
data2['use1'] = [formatted_sentences(text) if isinstance(text, str) else text for text in data2['use1']]
data2['use2'] = [formatted_sentences(text) if isinstance(text, str) else text for text in data2['use2']]
data2['use3'] = [formatted_sentences(text) if isinstance(text, str) else text for text in data2['use3']]
data2['use4'] = [formatted_sentences(text) if isinstance(text, str) else text for text in data2['use4']]

In [ ]:
data2['use0'].value_counts()

,count
use0,
bacterial infections,44053
pain relief,20578
gastroesophageal reflux disease (acid reflux),10867
type 2 diabetes mellitus,10099
hypertension (high blood pressure),10088
...,...
hairy cell leukemia,1
pruritus,1
localized muscle spasm,1


## Feature Engineering

### Adding Relevant Columns

Creating a new columns on data 1 which are :


*   type
*   primary_composition
*   all_composition
*   size_composition
*   all_size



In [ ]:
# Dataset az

def getType(row):
    for type in form:
        if type in row['pack_size_label'].lower():
            return type
def getComp(row):
    return row['short_composition1'].split('(')[0].lower().rstrip()

def getValue(row):
    return row['short_composition1'].split('(')[1].lower().rstrip().rstrip(')')


data1['type'] = data1.apply(lambda row: getType(row), axis=1)
data1['primary_composition'] = data1.apply(lambda row: getComp(row), axis=1)
data1['all_composition'] = data1['primary_composition'] + ' ' + np.where(data1['short_composition2'].isna(), '',
                                                         data1['short_composition2'].astype(str).apply(lambda x: x.split('(')[0].lower().rstrip().replace(' ', '')))
data1['size_composition'] = data1.apply(lambda row: getValue(row), axis=1)
data1['all_size'] =  data1['size_composition'] + ' ' + np.where(data1['short_composition2'].isna(), '',
                                                   data1['short_composition2'].astype(str).apply(lambda x: x.split('(')[1].lower().rstrip().rstrip(')') if len(x.split('(')) > 1 else ''))

data1.head()

,name,price(₹),Is_discontinued,manufacturer_name,type,pack_size_label,short_composition1,short_composition2,primary_composition,all_composition,size_composition,all_size
0,Augmentin 625 Duo Tablet,223.42,False,Glaxo SmithKline Pharmaceuticals Ltd,tablet,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg),amoxycillin,amoxycillin clavulanicacid,500mg,500mg 125mg
1,Azithral 500 Tablet,132.36,False,Alembic Pharmaceuticals Ltd,tablet,strip of 5 tablets,Azithromycin (500mg),NaN,azithromycin,azithromycin,500mg,500mg
2,Ascoril LS Syrup,118.00,False,Glenmark Pharmaceuticals Ltd,syrup,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml),ambroxol,ambroxol levosalbutamol,30mg/5ml,30mg/5ml 1mg/5ml
3,Allegra 120mg Tablet,218.81,False,Sanofi India Ltd,tablet,strip of 10 tablets,Fexofenadine (120mg),NaN,fexofenadine,fexofenadine,120mg,120mg
4,Avil 25 Tablet,10.96,False,Sanofi India Ltd,tablet,strip of 15 tablets,Pheniramine (25mg),NaN,pheniramine,pheniramine,25mg,25mg


Creating a new columns on data 2 which are :


*   main_substitution
*   other_substitution
*   side_effect
*   usage



In [ ]:
data2['main_substitution'] = data2.apply(lambda row: row['substitute0'] if not pd.isna(row['substitute0']) else '', axis=1)

data2['other_substitution'] = data2.apply(lambda row: ', '.join(set([str(row['substitute{}'.format(i)]) for i in range(1, 5)\
                                                     if not pd.isna(row['substitute{}'.format(i)])])), axis=1)

data2['side_effect'] = data2.apply(lambda row: ', '.join(set([str(row['sideEffect{}'.format(i)]) for i in range(42)\
                                                     if not pd.isna(row['sideEffect{}'.format(i)])])), axis=1)

data2['usage'] = data2.apply(lambda row: ' '.join([str(row['use{}'.format(i)]).replace(' ', '') for i in range(5) \
                                             if not pd.isna(row['use{}'.format(i)])]), axis=1)
data2.head()

,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,sideEffect2,sideEffect3,...,use3,use4,Chemical Class,Habit Forming,Therapeutic Class,Action Class,main_substitution,other_substitution,side_effect,usage
996,Alto's citrasol liquid,Cadicitron 1.4gm/5ml Syrup,Acrocitron 1.4gm/5ml Syrup,Alkacit Syrup,Alkawin 1.4gm/5ml Syrup,Citodac Syrup,Stomach pain,Tiredness,Diarrhea,Nausea,...,NaN,NaN,Carboxylic acid derivative,No,UROLOGY,Uricosuric agent-gout,Cadicitron 1.4gm/5ml Syrup,"Acrocitron 1.4gm/5ml Syrup, Alkawin 1.4gm/5ml ...","Tiredness, Diarrhea, Frequent urge to urinate,...",gout kidneystone
6434,a 1 5mg tablet,StayHappi Amlodipine 5mg Tablet,Amlip 5 Tablet,Avacard 5mg Tablet,Amcard 5 Tablet,Amlong Tablet,Headache,Fatigue,Nausea,Abdominal pain,...,NaN,NaN,Dihydropyridinecarboxylic acids derivatives,No,CARDIAC,Calcium channel blockers- Dihydropyridines (DHP),StayHappi Amlodipine 5mg Tablet,"Amcard 5 Tablet, Amlip 5 Tablet, Amlong Tablet...","Abdominal pain, Headache, Fatigue, Nausea, Sle...",hypertension(highbloodpressure) angina(heart-r...
6139,a 250 suspension,Rakbi 250mg Suspension,Alcocin Suspension,Tempros Suspension,Mormol 250mg Suspension,Jetmol Suspension,Indigestion,Stomach pain,Nausea,Vomiting,...,NaN,NaN,P-Aminophenol Derivative,No,PAIN ANALGESICS,Analgesic & Antipyretic-PCM,Rakbi 250mg Suspension,"Tempros Suspension, Jetmol Suspension, Mormol ...","Indigestion, Vomiting, Nausea, Stomach pain",painrelief fever
13016,a 3 100 mg/500 mg tablet,Nimsa Plus 100mg/500mg Tablet,Nicet Tablet,Nesu P 100mg/500mg Tablet,Nimeson P 100 mg/500 mg Tablet,Tromanil Plus 100 mg/500 mg Tablet,Nausea,Vomiting,Diarrhea,Increased liver enzymes,...,NaN,NaN,NaN,No,PAIN ANALGESICS,NaN,Nimsa Plus 100mg/500mg Tablet,"Nesu P 100mg/500mg Tablet, Nimeson P 100 mg/50...","Diarrhea, Vomiting, Increased liver enzymes, N...",painrelief
14692,a arti 60mg injection,Arteross 60mg Injection,Falciart 60mg Injection,Falsitis AR Injection,Leonate 60mg Injection,Neosunate 60mg Injection,Headache,Dizziness,Weakness,Loss of appetite,...,NaN,NaN,Sesquiterpene lactones,No,ANTI MALARIALS,Antimalarial- Artemisinin and derivatives,Arteross 60mg Injection,"Falsitis AR Injection, Neosunate 60mg Injectio...","Loss of appetite, Weakness, Injection site rea...",malaria


### Merging and evaluate data

In [ ]:
# Doing a left join merge between data 1 and data 2 by the column 'name'
df = pd.merge(data1, data2, left_on=data1['name'].str.lower(), right_on='name', how='left', indicator=True)

# Checks rows of data 1 that has no partner data
unmatched_rows = df[df['_merge'] == 'left_only']
percentage_unmatched = (len(unmatched_rows) / len(df)) * 100
print(f'Percentage of unmacthed rows: {percentage_unmatched:.2f}%')
print(f'Total rows of data 1 that has no partner with data 2: {len(unmatched_rows)}')

Percentage of unmacthed rows: 11.06%
Total rows of data 1 that has no partner with data 2: 28092


In [ ]:
# Dataset final that will be used on feature embedding
df_final = df[df.Is_discontinued==False]
df_final = df_final[['name', 'type', 'primary_composition',	'all_composition',
                     'size_composition',	'all_size',	'usage']]
df_final.head()

,name,type,primary_composition,all_composition,size_composition,all_size,usage
0,augmentin 625 duo tablet,tablet,amoxycillin,amoxycillin clavulanicacid,500mg,500mg 125mg,bacterialinfections
1,azithral 500 tablet,tablet,azithromycin,azithromycin,500mg,500mg,bacterialinfections
2,ascoril ls syrup,syrup,ambroxol,ambroxol levosalbutamol,30mg/5ml,30mg/5ml 1mg/5ml,coughwithmucus
3,allegra 120mg tablet,tablet,fexofenadine,fexofenadine,120mg,120mg,sneezingandrunnynoseduetoallergies allergiccon...
4,avil 25 tablet,tablet,pheniramine,pheniramine,25mg,25mg,allergicconditions


In [ ]:
# Rechecking missing value
missing_values = df_final.isnull().sum()
percentage_missing = (missing_values / len(df_final)) * 100

missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': percentage_missing})
print(missing_data)

                     Missing Values  Percentage
name                              0    0.000000
type                              0    0.000000
primary_composition               0    0.000000
all_composition                   0    0.000000
size_composition                  0    0.000000
all_size                          0    0.000000
usage                         27456   11.158118


In [ ]:
# Drop the missing values
df_final = df_final.dropna(subset=['usage'])

In [ ]:
# Specifies which columns will be processed for the model training.
text_columns = ['type','primary_composition', 'all_composition','size_composition','all_size','usage']

# Model Training

In [ ]:
df_fin = df_final.sample(frac=0.1, random_state=42)

In [ ]:
# Function to train the model on the sentences input by finding patterns in data
def generate_word_embeddings(column):
    unique_values = df_fin[column].unique()
    sentences = [[value] for value in unique_values]
    model = Word2Vec(sentences, min_count=1, vector_size=100)
    return model

In [ ]:
# A placeholder to hold the models
word_embedding_models = {}

for column in text_columns:
    model = generate_word_embeddings(column)
    word_embedding_models[column] = model

In [ ]:
# Applying the trained Word2Vec models to the original columns to create new columns that contain the embeddings
for column, model in tqdm(word_embedding_models.items(), desc="embeddings"):
    df_fin[column + '_embeddings'] = df_fin[column].apply(lambda x: np.round(model.wv[x].reshape(1, -1), 4) if x in model.wv else [])

embeddings: 100%|██████████| 6/6 [00:01<00:00,  5.18it/s]


In [ ]:
df_fin.head()

,name,type,primary_composition,all_composition,size_composition,all_size,usage,type_embeddings,primary_composition_embeddings,all_composition_embeddings,size_composition_embeddings,all_size_embeddings,usage_embeddings
30797,bdpoma 1mg capsule,capsule,pomalidomide,pomalidomide,1mg,1mg,multiplemyeloma,"[[-0.0012, -0.0024, -0.0018, -0.0012, -0.0065,...","[[-0.0035, -0.0031, 0.0051, 0.01, -0.0067, -0....","[[-0.0021, -0.0054, -0.0089, -0.0015, -0.0076,...","[[-0.0034, 0.0052, 0.003, 0.0017, 0.0007, -0.0...","[[-0.0006, -0.007, -0.0011, 0.009, -0.0057, 0....","[[0.0078, -0.0042, 0.0051, -0.0056, 0.0031, -0..."
174986,prestoheal ds tablet,tablet,bromelain,bromelain rutoside,180mg,180mg 200mg,painrelief,"[[-0.0025, -0.0059, 0.0075, -0.0073, -0.009, -...","[[-0.0025, -0.0096, -1e-04, 0.0019, 0.0002, 0....","[[0.0092, -0.009, -0.0066, -0.0075, 0.0092, -0...","[[0.0035, 0.0077, 0.0059, 0.0085, 0.0064, 0.00...","[[-0.0084, 0.0054, -0.006, 0.0024, -0.0008, 0....","[[0.0077, -0.0038, 0.0073, 0.007, 0.0036, -0.0..."
55160,cadiceft 250mg injection,injection,ceftriaxone,ceftriaxone,250mg,250mg,bacterialinfections,"[[-0.0002, 0.0042, 0.0021, 0.01, 0.0006, -0.00...","[[0.0053, 0.0049, 0.0058, -0.002, 0.0075, -0.0...","[[0.0019, 0.0067, -0.003, -0.0088, 0.0017, -0....","[[-0.0015, 0.0046, 0.0016, 0.0061, 0.0006, -0....","[[-0.0036, -0.0005, -0.0039, 0.009, -0.0012, -...","[[0.0048, 0.0021, -0.0051, -0.009, -0.0064, -0..."
221255,toplift xl 37.5mg capsule sr,capsule,venlafaxine,venlafaxine,37.5mg,37.5mg,depression anxietydisorder,"[[-0.0012, -0.0024, -0.0018, -0.0012, -0.0065,...","[[0.0014, -0.0034, 0.007, -0.0014, 0.0078, -0....","[[0.0077, -0.0043, 0.0014, -0.0077, -0.0057, 0...","[[-0.0072, -0.0093, -0.0039, 0.0094, -0.0039, ...","[[-0.0054, 0.0078, -0.0086, -0.0081, 0.0034, -...","[[-0.0068, -0.0069, -0.0093, 0.003, 0.0016, 0...."
132807,laurunam 1000mg injection,injection,meropenem,meropenem,1000mg,1000mg,severebacterialinfections,"[[-0.0002, 0.0042, 0.0021, 0.01, 0.0006, -0.00...","[[0.0028, -0.0095, -0.004, -0.0092, 0.002, -0....","[[-0.0039, 0.003, -0.0038, -0.0042, -0.0031, 0...","[[0.0079, 0.0097, 0.0004, 0.0015, 0.0032, -0.0...","[[-0.0043, -0.0049, -0.0023, 0.0012, 0.0043, 0...","[[-0.0092, -0.0094, -0.0025, 0.0012, 0.0017, 0..."


In [ ]:
# Determining the weights of the word from columns

weightage = {
    'type_embeddings': 0.1,               # Weight for the 'type' feature
    'primary_composition_embeddings': 0.1,       # Weight for the 'primary component' feature
    'all_composition_embeddings': 0.3,         # Weight for the 'entire component' feature
    'size_composition_embeddings': 0.1,               # Weight for the 'value' feature
    'all_size_embeddings': 0.1,        # Weight for the 'entire value' feature
    'usage_embeddings': 0.3                   # Weight for the 'use' feature
}

In [ ]:
# Calculate the similarities between 2 columns value
def calculate_text_similarity(value1, value2):
    similarity = cosine_similarity(value1, value2)
    return similarity[0][0]

# Calculate the weighted similarity between 2 medicines
def calculate_weighted_similarity(medicine1, medicine2):
    similarity_scores = []
    for column, weight in weightage.items():
        similarity = calculate_text_similarity(medicine1[column].values[0], medicine2[column])
        similarity_scores.append(similarity * weight)
    weighted_similarity = sum(similarity_scores)
    return weighted_similarity

In [ ]:
def get_similar_medicines(medicine_name):
    medicine = df_fin[df_fin['name'] == medicine_name]
    if medicine.empty:
        return "Medicine not found"

    if medicine['primary_composition'].values[0] == medicine['all_composition'].values[0].rstrip():
        del weightage['primary_composition_embeddings']

    if medicine['size_composition'].values[0] == medicine['all_size'].values[0].rstrip():
        del weightage['size_composition_embeddings']

    new_dataset = df_fin.copy(deep=True)
    new_dataset['similarity_score'] = df_fin.apply(lambda x: calculate_weighted_similarity(medicine, x), axis=1)
    top_10_similar_medicines = new_dataset.nlargest(11, 'similarity_score')[1:]
    return top_10_similar_medicines[['name', 'similarity_score']]

In [ ]:
df_fin['name']

,name
30797,bdpoma 1mg capsule
174986,prestoheal ds tablet
55160,cadiceft 250mg injection
221255,toplift xl 37.5mg capsule sr
132807,laurunam 1000mg injection
...,...
150512,newtel-h 40 tablet
64771,dolzy mr 250mg/50mg/325mg tablet
234242,valirab-dsr capsule
68692,dize 5mg tablet


# Model Testing

In [ ]:
similar_medicines = get_similar_medicines("laurunam 1000mg injection")
print('Top 10 Similar Medicine')
print(similar_medicines)

Top 10 Similar Medicine
                              name  similarity_score
148290     meropect 1000 injection               0.8
148695   merosler 1000mg injection               0.8
142276   my penum 1000mg injection               0.8
149475    mepwell 1000mg injection               0.8
10342     aspenem 1000mg injection               0.8
182804    peronum 1000mg injection               0.8
141544          meroclay injection               0.8
28829     biopect 1000mg injection               0.8
149625   merogoog 1000mg injection               0.8
146275  meroclass 1000mg injection               0.8


In [ ]:
similar_medicines = get_similar_medicines("prestoheal ds tablet")
print('Top 10 Similar Medicine')
print(similar_medicines)

Top 10 Similar Medicine
                                     name  similarity_score
140360       maczo 90mg/100mg/48mg tablet          0.708741
74598     edeoflam 90mg/100mg/48mg tablet          0.708741
64138   dru 100mg/325mg/50000iu tablet xl          0.474300
101038                  gettifen-p tablet          0.474300
27164                    bromenz-d tablet          0.469255
41836            chymotech br plus tablet          0.469255
71585        defidrot p 80mg/325mg tablet          0.468724
173839              proxidom 500mg tablet          0.466324
107941         ibumax 50 mg/500 mg tablet          0.463604
28692           bemol p 50mg/500mg tablet          0.463604
